# Finding the Ground State Energy of Molecular Hydrogen (H2) via the Variational Quantum Eigensolver (VQE).

VQE is the most commonly used algorithm to obtain the ground state energy of molecules due to lower requirements in the coherence times of the Quantum Processing Unit (QPU). 

As explained in the theory chapter it combines the quantum and classical processes. Specifically, it will use the QPU to measure the expectation value of the Molecular Hamiltonian ($\langle \Psi | H | \Psi \rangle$) and a classical optimization method to vary the parameters of the Wavefunction (commonly used optimizers are SLSQP or SPSA).

Let us see how do we obtain the Ground State Energy of H2 using Qiskit.

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature import settings

settings.use_pauli_sum_op = False


driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

The code above has used a classical Quantum Chemical Software (PySCF) to obtain the Hartree-Fock wavefunction.
We can easily visualize the matrix elements of the Molecular Hamiltionian by accessing the properties of the object `es_problem`

In [ ]:
ham_matrix = es_problem.hamiltonian.second_q_op()
print(ham_matrix)

In order to transform this problem into something that the QPU can operate with, it is needed to change basis from the Fock Space to Pauli basis. The specific transformation is called mapper or encoder. Let's use one of the more extended mappers as is the `JordanWignerMapper`

In [ ]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

And now, let us define the solver for finding the lowest eigenvalue of the Hamiltonian with respect to the parametrized wavefunction. At this point we also need to define the quantum circuit that we want to use. In this case, since the molecule and basis set are small, we can opt for a chemically ispired ansatz as Unitary Coupled-Cluster (UCC).

In [ ]:
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

initial_state = HartreeFock(es_problem.num_spatial_orbitals,es_problem.num_particles,mapper)

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=initial_state
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

Once we have defined all the necessary ingredients for obtaining the ground state energy we can use the overall algorithm `GroundStateEigensolver` that wraps all the above defined variables and algos into a single and compact algorithm.

In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(es_problem)
print(res)